In [1]:
using CSV, DataFrames, GLM, Statistics, StatsModels, Plots

Cargamos los datos

In [28]:
file_path = "bottle.csv"
df = CSV.read(file_path, DataFrame);

### Función `dataType`, me da el tipo de dato por columna

In [4]:
function dataType(df::DataFrame)
    clean_df = dropmissing(df, names(df))
    for i = 1:ncol(df)
        column_name = names(df)[i]
        column_type = eltype(clean_df[:, column_name])
        println("Data Type column $column_name: $column_type")
    end
end

dataType(df)

Data Type column Cst_Cnt: Int64
Data Type column Btl_Cnt: Int64
Data Type column Sta_ID: String15
Data Type column Depth_ID: String
Data Type column Depthm: Int64
Data Type column T_degC: Float64
Data Type column Salnty: Float64
Data Type column O2ml_L: Float64
Data Type column STheta: Float64
Data Type column O2Sat: Float64
Data Type column Oxy_µmol/Kg: Float64
Data Type column BtlNum: Int64
Data Type column RecInd: Int64
Data Type column T_prec: Int64
Data Type column T_qual: Int64
Data Type column S_prec: Int64
Data Type column S_qual: Int64
Data Type column P_qual: Int64
Data Type column O_qual: Int64
Data Type column SThtaq: Int64
Data Type column O2Satq: Int64
Data Type column ChlorA: Float64
Data Type column Chlqua: Int64
Data Type column Phaeop: Float64
Data Type column Phaqua: Int64
Data Type column PO4uM: Float64
Data Type column PO4q: Int64
Data Type column SiO3uM: Float64
Data Type column SiO3qu: Int64
Data Type column NO2uM: Float64
Data Type column NO2q: Int64
Data Type c

### Función `count_missing`, cuenta el número de datos faltantes en la columna dada

In [5]:
function count_missing(df::DataFrame, column_name)
        missing_count = count(ismissing, df[:, column_name])
        println("En la columna $column_name hay $missing_count datos faltantes")
end

count_missing(df,"T_degC")
count_missing(df,"R_CHLA")

En la columna T_degC hay 10963 datos faltantes
En la columna R_CHLA hay 639587 datos faltantes


### Función `dataMissingPercentage()`, itera sobre las columnas y obtiene el porcentaje de datos faltantes

In [7]:
function dataMissingPercentage(df::DataFrame)
    for i = 1:ncol(df)
        column_name = names(df)[i]
        missing_count = count(ismissing, df[:, column_name])
        missing_percentage = (missing_count / nrow(df)) * 100
        println("En la columna $column_name falta el ($missing_percentage%) de los datos")
    end
end

dataMissingPercentage(df)

En la columna Cst_Cnt falta el (0.0%) de los datos
En la columna Btl_Cnt falta el (0.0%) de los datos
En la columna Sta_ID falta el (0.0%) de los datos
En la columna Depth_ID falta el (0.0%) de los datos
En la columna Depthm falta el (0.0%) de los datos
En la columna T_degC falta el (1.2675996082616552%) de los datos
En la columna Salnty falta el (5.475318056154559%) de los datos
En la columna O2ml_L falta el (19.501585800294382%) de los datos
En la columna STheta falta el (6.092178761260454%) de los datos
En la columna O2Sat falta el (23.54002888318728%) de los datos
En la columna Oxy_µmol/Kg falta el (23.54072263468318%) de los datos
En la columna BtlNum falta el (86.27909853930622%) de los datos
En la columna RecInd falta el (0.0%) de los datos
En la columna T_prec falta el (1.2675996082616552%) de los datos
En la columna T_qual falta el (97.32593485904704%) de los datos
En la columna S_prec falta el (5.475318056154559%) de los datos
En la columna S_qual falta el (91.33805007267047%

### Función `deleteColumns()` elimina todas las columnas que no superen el umbral de completitud dado por el `threshold`

In [8]:
function deleteColumns(df, completeness_threshold)
    clean_df = dropmissing(df, names(df))
    num_rows = nrow(df)
    valid_columns = String[]
    for col_name in names(df)
        col = df[:, col_name]
        if count(ismissing, col) / num_rows <= (1 - completeness_threshold)
            push!(valid_columns, string(col_name))  # Asegurar que son nombres en formato String
        end
    end
    df_subset = select(df, intersect(names(df), valid_columns))
    return df_subset
end

deleteColumns(df, 1.0)

864863×8 DataFrame
    Row │ Cst_Cnt  Btl_Cnt  Sta_ID       Depth_ID                           De ⋯
        │ Int64    Int64    String15     String                             In ⋯
────────┼───────────────────────────────────────────────────────────────────────
      1 │       1        1  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…     ⋯
      2 │       1        2  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…
      3 │       1        3  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…
      4 │       1        4  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…
      5 │       1        5  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…     ⋯
      6 │       1        6  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…
      7 │       1        7  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…
      8 │       1        8  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…
      9 │       1        9  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…     ⋯
     10 │       1       10  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…
     11 │       1       11  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…
   ⋮    │    ⋮        ⋮          ⋮                       ⋮                     ⋱
 864854 │   34403   864854  093.3 120.0  20-1611SR-MX-313-2053-09331200-0…
 864855 │   34403   864855  093.3 120.0  20-1611SR-MX-313-2053-09331200-0…     ⋯
 864856 │   34403   864856  093.3 120.0  20-1611SR-MX-313-2053-09331200-0…
 864857 │   34403   864857  093.3 120.0  20-1611SR-MX-313-2053-09331200-0…
 864858 │   34403   864858  093.3 120.0  20-1611SR-MX-313-2053-09331200-0…
 864859 │   34404   864859  093.4 026.4  20-1611SR-MX-310-2239-09340264-0…     ⋯
 864860 │   34404   864860  093.4 026.4  20-1611SR-MX-310-2239-09340264-0…
 864861 │   34404   864861  093.4 026.4  20-1611SR-MX-310-2239-09340264-0…
 864862 │   34404   864862  093.4 026.4  20-1611SR-MX-310-2239-09340264-0…
 864863 │   34404   864863  093.4 026.4  20-1611SR-MX-310-2239-09340264-0…     ⋯
                                               4 columns and 864842 rows omitted

### Función `calculate_correlation`, calcula la matriz de correlación

Para este calculo se diseñó la función `filter_columns` que es similar a `deleteColumns`, pero ahora no solo elimina las columnas que no pasen cierto umbral de completitud, sino también las que no tengan datos numéricos.

También se tiene la función `clean_dataframe` que basado en las columnas filtradas, genera un nuevo DataFrame para despues poder usar `calculate_correlation`

In [21]:
function filter_columns(df, completeness_threshold)
    clean_df = dropmissing(df, names(df))
    num_rows = nrow(df)
    valid_columns = String[]
    for col_name in names(df)
        col = df[:, col_name]
        col_type = clean_df[:, col_name]
        if eltype(col_type) <: Number && count(ismissing, col) / num_rows <= (1 - completeness_threshold)
            push!(valid_columns, string(col_name))  # Asegurar que son nombres en formato String
        end
    end
    return valid_columns
end

function clean_dataframe(df::DataFrame, valid_columns::Vector{String})
    # Seleccionar solo las columnas válidas
    df_subset = select(df, intersect(names(df), valid_columns))
    return df_subset
end


function calculate_correlation(df::DataFrame)
    # Eliminar filas con valores faltantes
    clean_df = dropmissing(df)
    
    # Seleccionar columnas numéricas
    numeric_cols = names(clean_df)[eltype.(eachcol(clean_df)) .<: Number]
    
    # Inicializar matriz de correlaciones
    corr_mat = Dict{String, Dict{String, Float64}}()
    for col1 in numeric_cols
        corr_mat[string(col1)] = Dict(string(col2) => 0.0 for col2 in numeric_cols)
    end
    
    # Calcular las correlaciones
    for i in 1:length(numeric_cols)
        for j in i:length(numeric_cols)
            col1 = numeric_cols[i]
            col2 = numeric_cols[j]
            
            # Seleccionar filas sin valores faltantes para ambas columnas
            valid_rows = .!(ismissing.(df[!, col1]) .| ismissing.(df[!, col2]))
            
            # Si no hay datos suficientes, omitir
            if sum(valid_rows) <= 1
                correlation = NaN
            else
                m1 = mean(df[valid_rows, col1])
                m2 = mean(df[valid_rows, col2])
                cov = mean((df[valid_rows, col1] .- m1) .* (df[valid_rows, col2] .- m2))
                std1 = std(df[valid_rows, col1])
                std2 = std(df[valid_rows, col2])
                correlation = cov / (std1 * std2)
            end
            
            # Actualizar la matriz
            corr_mat[string(col1)][string(col2)] = correlation
            corr_mat[string(col2)][string(col1)] = correlation
        end
    end
    
    return corr_mat
end
    
valid_cols = filter_columns(df, 0.93)
df_subset = clean_dataframe(df,valid_cols);
corr_mat = calculate_correlation(df_subset)

Dict{String, Dict{String, Float64}} with 22 entries:
  "C14A2q"     => Dict("C14A2q"=>0.999999, "R_SVA"=>-0.00497251, "NH3q"=>-0.000…
  "R_SVA"      => Dict("C14A2q"=>-0.00497251, "R_SVA"=>0.999999, "NH3q"=>-0.006…
  "NH3q"       => Dict("C14A2q"=>-0.00030135, "R_SVA"=>-0.00611033, "NH3q"=>0.9…
  "Cst_Cnt"    => Dict("C14A2q"=>-0.00306502, "R_SVA"=>0.107822, "NH3q"=>-0.191…
  "R_SALINITY" => Dict("C14A2q"=>0.00407469, "R_SVA"=>-0.698362, "NH3q"=>0.0252…
  "S_prec"     => Dict("C14A2q"=>-0.00187998, "R_SVA"=>-0.00640179, "NH3q"=>-0.…
  "STheta"     => Dict("C14A2q"=>0.00426926, "R_SVA"=>-0.863455, "NH3q"=>0.0053…
  "R_TEMP"     => Dict("C14A2q"=>-0.00416906, "R_SVA"=>0.96417, "NH3q"=>0.00099…
  "RecInd"     => Dict("C14A2q"=>0.00263151, "R_SVA"=>-0.0663313, "NH3q"=>0.016…
  "T_degC"     => Dict("C14A2q"=>-0.00416907, "R_SVA"=>0.964169, "NH3q"=>0.0010…
  "R_Depth"    => Dict("C14A2q"=>0.00198313, "R_SVA"=>-0.669229, "NH3q"=>0.0115…
  "Depthm"     => Dict("C14A2q"=>0.00198312, "R_SVA"=>-0

### Función `display_correlation()`, esta función usa la matriz de correlación creada en `display_correlation()` y crea un heatmap

In [22]:
function display_correlation(corr_mat::Dict)
    # Extraer nombres de filas y columnas
    rows = collect(keys(corr_mat))
    cols = collect(keys(first(values(corr_mat))))
    
    # Crear la matriz de correlaciones
    matrix = [corr_mat[row][col] for row in rows, col in cols]
    
    # Crear el heatmap
    fig = heatmap(
        cols,       # Las columnas (variables)
        rows,       # Las filas (también variables)
        matrix,     # Los valores de correlación
        color=:viridis,
        title="Heatmap de Correlación",
        xlabel="Variables",
        ylabel="Variables",
        size=(800, 600),
        colorbar=true  # Activar la barra de colores
    )
    
    # Guardar la imagen del heatmap
    savefig("Correlation_Matrix.png")
    return fig
end

display_correlation(corr_mat)

### Función `removeOutliersIQR()`, elimina los valores atípicos (outliers) de las columnas numéricas en un DataFrame

In [23]:
function removeOutliersIQR(clean_df::DataFrame)
    for col in names(clean_df)
        if eltype(clean_df[!, col]) <: Number
            Q1 = quantile(skipmissing(clean_df[!, col]), 0.25)
            Q3 = quantile(skipmissing(clean_df[!, col]), 0.75)
            IQR = Q3 - Q1
            
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            # Filtrar filas dentro del rango
            clean_df = filter(row -> row[col] ≥ lower_bound && row[col] ≤ upper_bound, clean_df)
        end
    end
    return clean_df
end

removeOutliersIQR(df_subset)

790980×22 DataFrame
    Row │ Cst_Cnt  Btl_Cnt  Depthm  T_degC    Salnty    STheta    RecInd  T_pr ⋯
        │ Int64    Int64    Int64   Float64?  Float64?  Float64?  Int64   Int6 ⋯
────────┼───────────────────────────────────────────────────────────────────────
      1 │       1        1       0    10.5     33.44     25.649        3       ⋯
      2 │       1        2       8    10.46    33.44     25.656        3
      3 │       1        3      10    10.46    33.437    25.654        7
      4 │       1        4      19    10.45    33.42     25.643        3
      5 │       1        5      20    10.45    33.421    25.643        7       ⋯
      6 │       1        6      30    10.45    33.431    25.651        7
      7 │       1        7      39    10.45    33.44     25.658        3
      8 │       1        8      50    10.24    33.424    25.682        7
      9 │       1        9      58    10.06    33.42     25.71         3       ⋯
     10 │       1       10      75     9.86    33.494    25.801        7
     11 │       1       11      78     9.83    33.51     25.819        3
   ⋮    │    ⋮        ⋮       ⋮        ⋮         ⋮         ⋮        ⋮       ⋮  ⋱
 790971 │   34403   864854     381     6.943   34.1104   26.7357       3
 790972 │   34403   864855     400     6.694   34.1101   26.7693       7       ⋯
 790973 │   34403   864856     440     6.312   34.1563   26.8564       3
 790974 │   34403   864857     500     5.993   34.216    26.9452       7
 790975 │   34403   864858     521     5.818   34.2382   26.9848       3
 790976 │   34404   864859       0    18.744   33.4083   23.8706       7       ⋯
 790977 │   34404   864860       2    18.744   33.4083   23.8707       3
 790978 │   34404   864861       5    18.692   33.415    23.8891       3
 790979 │   34404   864862      10    18.161   33.4062   24.0143       3
 790980 │   34404   864863      15    17.533   33.388    24.153        3       ⋯
                                              15 columns and 790959 rows omitted

### Función `deleteRow()`, dada una columna `column_name`, se limpia el dataframe eliminando todas las filas que tenga incompletas esa columna

In [25]:
function deleteRow(df::DataFrame, column_name)
    # Filtrar las filas donde los valores de la columna no sean nulos
    clean_df = filter(row -> !ismissing(row[column_name]), df)
    return clean_df
end

deleteRow(df_subset, "T_degC")

853900×22 DataFrame
    Row │ Cst_Cnt  Btl_Cnt  Depthm  T_degC    Salnty    STheta    RecInd  T_pr ⋯
        │ Int64    Int64    Int64   Float64?  Float64?  Float64?  Int64   Int6 ⋯
────────┼───────────────────────────────────────────────────────────────────────
      1 │       1        1       0    10.5     33.44     25.649        3       ⋯
      2 │       1        2       8    10.46    33.44     25.656        3
      3 │       1        3      10    10.46    33.437    25.654        7
      4 │       1        4      19    10.45    33.42     25.643        3
      5 │       1        5      20    10.45    33.421    25.643        7       ⋯
      6 │       1        6      30    10.45    33.431    25.651        7
      7 │       1        7      39    10.45    33.44     25.658        3
      8 │       1        8      50    10.24    33.424    25.682        7
      9 │       1        9      58    10.06    33.42     25.71         3       ⋯
     10 │       1       10      75     9.86    33.494    25.801        7
     11 │       1       11      78     9.83    33.51     25.819        3
   ⋮    │    ⋮        ⋮       ⋮        ⋮         ⋮         ⋮        ⋮       ⋮  ⋱
 853891 │   34403   864854     381     6.943   34.1104   26.7357       3
 853892 │   34403   864855     400     6.694   34.1101   26.7693       7       ⋯
 853893 │   34403   864856     440     6.312   34.1563   26.8564       3
 853894 │   34403   864857     500     5.993   34.216    26.9452       7
 853895 │   34403   864858     521     5.818   34.2382   26.9848       3
 853896 │   34404   864859       0    18.744   33.4083   23.8706       7       ⋯
 853897 │   34404   864860       2    18.744   33.4083   23.8707       3
 853898 │   34404   864861       5    18.692   33.415    23.8891       3
 853899 │   34404   864862      10    18.161   33.4062   24.0143       3
 853900 │   34404   864863      15    17.533   33.388    24.153        3       ⋯
                                              15 columns and 853879 rows omitted

### Función `filterColumnsByCorrelation(target,threshold, realtion)`, basado en el calculo de la correlación, se establece un threshold de correlación con una columna dada `target` y se crea un nuevo DataFrame con las columnas que cumplan dicho threshold

In [26]:
function filter_columns_by_correlation(df::DataFrame, corr_mat::Dict, target::Symbol, threshold::Float64, relation::Bool)
    target_corr = corr_mat[string(target)]
    
    for key in keys(target_corr)
        if key == string(target)
            continue
        end
        
        val = target_corr[key]
        if (!relation && abs(val) ≤ abs(threshold)) || (relation && val ≥ threshold)
            select!(df, Not(Symbol(key)))  # Eliminar columna
        end
    end
    
    return df
end

df1 = filter_columns_by_correlation(df_subset, corr_mat, :T_degC, 0.1, false)

864863×12 DataFrame
    Row │ Depthm  T_degC    Salnty    STheta    R_Depth  R_TEMP    R_POTEMP  R ⋯
        │ Int64   Float64?  Float64?  Float64?  Float64  Float64?  Float64?  F ⋯
────────┼───────────────────────────────────────────────────────────────────────
      1 │      0    10.5     33.44     25.649       0.0     10.5      10.5     ⋯
      2 │      8    10.46    33.44     25.656       8.0     10.46     10.46
      3 │     10    10.46    33.437    25.654      10.0     10.46     10.46
      4 │     19    10.45    33.42     25.643      19.0     10.45     10.45
      5 │     20    10.45    33.421    25.643      20.0     10.45     10.45    ⋯
      6 │     30    10.45    33.431    25.651      30.0     10.45     10.45
      7 │     39    10.45    33.44     25.658      39.0     10.45     10.45
      8 │     50    10.24    33.424    25.682      50.0     10.24     10.23
      9 │     58    10.06    33.42     25.71       58.0     10.06     10.05    ⋯
     10 │     75     9.86    33.494    25.801      75.0      9.86      9.85
     11 │     78     9.83    33.51     25.819      78.0      9.83      9.82
   ⋮    │   ⋮        ⋮         ⋮         ⋮         ⋮        ⋮         ⋮        ⋱
 864854 │    381     6.943   34.1104   26.7357    381.0      6.94      6.91
 864855 │    400     6.694   34.1101   26.7693    400.0      6.69      6.66    ⋯
 864856 │    440     6.312   34.1563   26.8564    440.0      6.31      6.27
 864857 │    500     5.993   34.216    26.9452    500.0      5.99      5.95
 864858 │    521     5.818   34.2382   26.9848    521.0      5.82      5.77
 864859 │      0    18.744   33.4083   23.8706      0.0     18.74     18.74    ⋯
 864860 │      2    18.744   33.4083   23.8707      2.0     18.74     18.74
 864861 │      5    18.692   33.415    23.8891      5.0     18.69     18.69
 864862 │     10    18.161   33.4062   24.0143     10.0     18.16     18.16
 864863 │     15    17.533   33.388    24.153      15.0     17.53     17.53    ⋯
                                               5 columns and 864842 rows omitted

### Función `describe(df)`

In [27]:
describe(df)

74×7 DataFrame
 Row │ variable             mean       min                                medi ⋯
     │ Symbol               Union…     Any                                Unio ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ Cst_Cnt              17138.8    1                                  1684 ⋯
   2 │ Btl_Cnt              432432.0   1                                  4324
   3 │ Sta_ID                          001.0 168.0                            
   4 │ Depth_ID                        19-4903CR-HY-060-0930-05400560-0…      
   5 │ Depthm               226.832    0                                  125. ⋯
   6 │ T_degC               10.7997    1.44                               10.0
   7 │ Salnty               33.8404    28.431                             33.8
   8 │ O2ml_L               3.39247    -0.01                              3.44
   9 │ STheta               25.8194    20.934                             25.9 ⋯
  10 │ O2Sat                57.1038    -0.1                               54.4
  11 │ Oxy_µmol/Kg          148.809    -0.4349                            151.
  ⋮  │          ⋮               ⋮                      ⋮                     ⋮ ⋱
  65 │ R_PHAEO              0.198599   -3.89                              0.11
  66 │ R_PRES               228.396    0                                  126. ⋯
  67 │ R_SAMP               162.072    0                                  206.
  68 │ DIC1                 2153.24    1948.85                            2170
  69 │ DIC2                 2168.15    1969.44                            2265
  70 │ TA1                  2256.06    2181.57                            2244 ⋯
  71 │ TA2                  2278.86    2198.15                            2247
  72 │ pH2                  7.94857    7.9231                             7.94
  73 │ pH1                  7.91098    7.6183                             7.92
  74 │ DIC Quality Comment             Bottle tripped at correct depth         ⋯
                                                   4 columns and 53 rows omitted